In [1]:
# Bibliotecas usadas
import os
import pandas as pd
import re 
from fuzzywuzzy import fuzz, process

m:\apps no sd\Python\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
covenants_notas = r"M:\Maria Eduarda\Downloads\Empresas Com Covenants"
dados = []

for caminho, subpastas, arquivos in os.walk(covenants_notas):
    if caminho == covenants_notas:
        continue
    
    relative_path = os.path.relpath(caminho, covenants_notas)
    empresa = relative_path.split(os.sep)[0]
    
    quantidade_anos = len([arquivo for arquivo in arquivos if arquivo.endswith(".pdf")])
    
    if quantidade_anos > 0:
        existente = next((d for d in dados if d["Empresa"] == empresa), None)
        if existente:
            existente["Quantidade de Anos"] += quantidade_anos
        else:
            dados.append({"Empresa": empresa, "Quantidade de Anos": quantidade_anos})
    
df = pd.DataFrame(dados)

df.to_excel("Empresas_com_covenants.xlsx", index=False)

display(df)

,Empresa,Quantidade de Anos
0,Hidrovias do Brasil,11
1,Ambipar Participacoes e Empreendimentos,4
2,Plano & Plano Desenvolvimento Imobiliario SA,4
3,Renner,13
4,B3 SA Brasil Bolsa Balcao,3
...,...,...
277,CTEEP - Companhia de Transmissao de Energia El...,13
278,Lwsa,4
279,BRF SA,6
280,Bradesco,2


In [14]:
df_anos = df
df_empresas_setores = pd.read_excel("Setores das empresas B3.xlsx")
df_empresas_setores.columns = df_empresas_setores.columns.str.strip()
lista_nomes = df_empresas_setores["Nome da empresa no pregão"].dropna().tolist()
df_empresas_setores

,Nome da empresa no pregão,Ticker na bolsa,Setor
0,AERIS,AERI3,Bens Industriais
1,ARMAC,ARML3,Bens Industriais
2,ATMASA,ATMP3,Bens Industriais
3,AZEVEDO,AZEV4,Bens Industriais
4,AZUL,AZUL4,Bens Industriais
...,...,...,...
370,POLPAR,PPAR3,Outros
371,PROMPT PART,PRPT3,Outros
372,SUDESTE S/A,OPSE3,Outros
373,SUL 116 PART,OPTS3,Outros


In [17]:
def match_nome(nome, lista_nomes, threshold=60):
    melhor_match = process.extractOne(nome, lista_nomes, scorer=fuzz.partial_ratio)
    if melhor_match and melhor_match[1] >= threshold:
        return melhor_match[0]
    return None

df_anos["Nome da empresa no pregão"] = df_anos["Empresa"].apply(
    lambda x: match_nome(x, lista_nomes)
)

df_merged = pd.merge(
    df_anos,
    df_empresas_setores,
    on="Nome da empresa no pregão",
    how="left"
)

df_merged.to_excel("empresas_merge_final.xlsx", index=False)

display(df_merged)

,Empresa,Quantidade de Anos,Nome da empresa,Nome da empresa no pregão,Ticker na bolsa,Setor
0,Hidrovias do Brasil,11,HIDROVIAS,HIDROVIAS,HBSA3,Bens Industriais
1,Ambipar Participacoes e Empreendimentos,4,AMBIPAR,AMBIPAR,AMBP3,Utilidade Pública
2,Plano & Plano Desenvolvimento Imobiliario SA,4,MOBLY,MOBLY,MBLY3,Consumo Cíclico
3,Renner,13,LOJAS RENNER,LOJAS RENNER,LREN3,Consumo Cíclico
4,B3 SA Brasil Bolsa Balcao,3,B3,B3,B3SA3,Financeiro
...,...,...,...,...,...,...
277,CTEEP - Companhia de Transmissao de Energia El...,13,ENERGISA,ENERGISA,ENGI11,Utilidade Pública
278,Lwsa,4,ESTRELA,ESTRELA,ESTR4,Consumo Cíclico
279,BRF SA,6,BRF SA,BRF SA,BRFS3,Consumo Não Cíclico
280,Bradesco,2,BRADESCO,BRADESCO,BBDC4,Financeiro
